In [17]:
import os
import re
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [18]:
# load data
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()

# remove sonnet numbers and convert to lowercase
text = re.sub(r'[0-9]+', '', text) 
text = text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

n_chars = len(text) # total length of dataset
n_vocab = len(chars) # number of unique characters

In [19]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 40
dataX = []
dataY = []

# use sliding window approach
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
n_sequences = len(dataX)

In [20]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_sequences, seq_length, 1))

# normalize data to range (0, 1)
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [21]:
# define the LSTM model 
# (single layer with 150 units, followed by dense output layer)
model = Sequential()
model.add(LSTM(200, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))

# start training from previous checkpoint
# file = "weights-improvement-20-1.7396.hdf5"
# model.load_weights(file)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [22]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
# train the model
model.fit(X, y, epochs=100, batch_size=8, callbacks=callbacks_list)

Epoch 1/100
97635/97635 [==============================] - 453s 5ms/step - loss: 2.6930

Epoch 00001: loss improved from inf to 2.69304, saving model to weights-improvement-01-2.6930.hdf5
Epoch 2/100
97635/97635 [==============================] - 454s 5ms/step - loss: 2.4843

Epoch 00002: loss improved from 2.69304 to 2.48432, saving model to weights-improvement-02-2.4843.hdf5
Epoch 3/100
97635/97635 [==============================] - 509s 5ms/step - loss: 2.3914

Epoch 00003: loss improved from 2.48432 to 2.39142, saving model to weights-improvement-03-2.3914.hdf5
Epoch 4/100
97635/97635 [==============================] - 437s 4ms/step - loss: 2.3108

Epoch 00004: loss improved from 2.39142 to 2.31076, saving model to weights-improvement-04-2.3108.hdf5
Epoch 5/100
97635/97635 [==============================] - 498s 5ms/step - loss: 2.2419

Epoch 00005: loss improved from 2.31076 to 2.24192, saving model to weights-improvement-05-2.2419.hdf5
Epoch 6/100
97635/97635 [===================